In [47]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pyodbc as po
from sqlalchemy import Table, MetaData, Column, Integer, DateTime

In [48]:
#Kết nối vào DDS
driver = "SQL Server Native Client 11.0"
server = "localhost"
database = "covid_dds"
con_str = f"DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes"
con_url = URL.create(drivername="mssql+pyodbc", query={"odbc_connect": con_str})
engine = create_engine(con_url)

In [49]:
#Lấy bảng fact vào dataframe
SQL_Query = "select * from fact_cases"
df = pd.read_sql_query(sql=SQL_Query, con=engine)
df

,phu_id,exposure_id,gender_id,age_group_id,date_id,total_cases,total_fatal_cases,total_resolved_cases,severity
0,131,15,11,21,162,1,0,1,1
1,131,15,11,22,92,1,0,1,1
2,131,15,11,22,171,1,0,1,1
3,131,15,11,22,275,1,0,1,1
4,131,15,11,23,137,1,0,1,1
...,...,...,...,...,...,...,...,...,...
215551,195,21,14,21,470,1,0,1,1
215552,195,21,14,21,474,1,0,1,1
215553,195,21,14,21,497,1,0,1,1
215554,195,21,14,21,504,1,0,1,1


In [50]:
#Xem danh sách số lượng dòng dữ liệu tương ứng với mỗi PHU
df["phu_id"].value_counts()

162    19915
149    18230
161    15215
148    10319
133    10052
       ...  
167       58
179       25
193       19
192       15
172       14
Name: phu_id, Length: 65, dtype: int64

In [51]:
#Đọc dữ liệu từ chiều Date vào 1 dataframe khác
df_time = pd.read_sql_query(sql="select id, day_in_month, month_in_year, [year] from dim_date", con=engine)
df_time

,id,day_in_month,month_in_year,year
0,1,1,1,2020
1,2,2,1,2020
2,3,3,1,2020
3,4,4,1,2020
4,5,5,1,2020
...,...,...,...,...
1091,1092,27,12,2022
1092,1093,28,12,2022
1093,1094,29,12,2022
1094,1095,30,12,2022


In [52]:
#Ghép ngày, tháng, năm trong bảng chiều Date hoàn chỉnh và trả kết quả vào 1 cột date
df_time["date"] = pd.to_datetime(df_time["year"].astype(str) + "-" 
                                    + df_time["month_in_year"].astype(str) + "-" 
                                    + df_time["day_in_month"].astype(str),
                                     yearfirst=True)
df_time

,id,day_in_month,month_in_year,year,date
0,1,1,1,2020,2020-01-01
1,2,2,1,2020,2020-01-02
2,3,3,1,2020,2020-01-03
3,4,4,1,2020,2020-01-04
4,5,5,1,2020,2020-01-05
...,...,...,...,...,...
1091,1092,27,12,2022,2022-12-27
1092,1093,28,12,2022,2022-12-28
1093,1094,29,12,2022,2022-12-29
1094,1095,30,12,2022,2022-12-30


In [53]:
#Tạo 1 cột date mới trong dataframe bảng fact, ánh xạ dữ liệu từ cột date mới hình thành trong dataframe bảng chiều Date qua dựa vào date_id
df["date"] = df["date_id"].map(df_time["date"])
df

,phu_id,exposure_id,gender_id,age_group_id,date_id,total_cases,total_fatal_cases,total_resolved_cases,severity,date
0,131,15,11,21,162,1,0,1,1,2020-06-11
1,131,15,11,22,92,1,0,1,1,2020-04-02
2,131,15,11,22,171,1,0,1,1,2020-06-20
3,131,15,11,22,275,1,0,1,1,2020-10-02
4,131,15,11,23,137,1,0,1,1,2020-05-17
...,...,...,...,...,...,...,...,...,...,...
215551,195,21,14,21,470,1,0,1,1,2021-04-15
215552,195,21,14,21,474,1,0,1,1,2021-04-19
215553,195,21,14,21,497,1,0,1,1,2021-05-12
215554,195,21,14,21,504,1,0,1,1,2021-05-19


In [54]:
#Sum các total_cases của PHU mã 162 theo ngày
sum = pd.DataFrame(df[df["phu_id"] == 162][["date", "total_cases"]].groupby(pd.Grouper(key="date", freq="D", axis=0)).sum()).reset_index(names=["date"])
sum

,date,total_cases
0,2020-01-22,1
1,2020-01-23,2
2,2020-01-24,0
3,2020-01-25,0
4,2020-01-26,0
...,...,...
494,2021-05-30,85
495,2021-05-31,67
496,2021-06-01,104
497,2021-06-02,93


In [57]:
#SQL to create PHU_162_stat
'''
create table PHU_162_stat
  (
	[date] datetime primary key,
	total_cases int
  )
'''

#Tạo bảng
table_name = "PHU_162_stat_date"
m = MetaData()
t = Table(table_name, m,
        Column("Date", DateTime, primary_key=True),
        Column("Total Cases", Integer))
m.create_all(engine)

In [58]:
#Xuất dữ liệu summary dataframe vào bảng vừa mới tạo
sum.to_sql(name=table_name, con=engine, index=False, if_exists="replace")

-1